<a href="https://colab.research.google.com/github/dentadelta/123/blob/master/Parallel%20Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import warnings
warnings.filterwarnings('ignore')

In [59]:
sentence1input = [1,2,3]
sentence2input = [4,5,6]
sentence1output = [7,8,9]
sentence2output = [10,11,12]
batch_input = [sentence1input, sentence2input]
batch_input = torch.tensor(batch_input)
print(batch_input)

tensor([[1, 2, 3],
        [4, 5, 6]])


In [60]:
h = 2 # 8 parallel attention layer
N = 6 # 6 stacks of encoder layer
dmodel = 512 #512 features
dk= int(dmodel/h) #key size, and value size
dv = int(dmodel/h)
dff = N*dmodel #embedding dimension
maximum_sequence_length = 3 #in real world in could be something like 128 pad pad sequence

In [61]:
num_embedding = len(sentence1input)+len(sentence2input)+len(sentence1output)+len(sentence2output)
print('Number of vocabulary size is:', num_embedding,'\n','In real life this number is as large as the vocabulary size - say 5000 unique words') 

Number of vocabulary size is: 12 
 In real life this number is as large as the vocabulary size - say 5000 unique words


In [62]:
# lets create an embedded layer:
embedded_layer = nn.Embedding(dff,dmodel)

In [63]:
# lets transform the batch input
batch_input = embedded_layer(batch_input)

In [64]:
#lets apply positional encoding
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [65]:
pe = PositionalEncoding(512,0.1,12)

In [66]:
batch_input = pe(batch_input)

In [124]:
# Lets create a self attention layer
class MultiAttentionHead(nn.Module):
  def __init__(self,dmodel,dk,dv,maximum_sequence_length):
    super(MultiAttentionHead, self).__init__()

    # I have two GPUs
    #Layer1: GPU:0
    self.keyTransformer_1 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_1 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_1  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer2: GPU:0
    self.keyTransformer_2 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_2 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_2  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer3: GPU:0
    self.keyTransformer_3 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_3 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_3  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer4: GPU:1
    self.keyTransformer_4 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_4 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_4  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer5: GPU:1
    self.keyTransformer_5 = nn.Linear(dmodel,dk).to('cuda:1')
    self.querryTransformer_5 = nn.Linear(dmodel,dk).to('cuda:1')
    self.valueTrasformer_5  = nn.Linear(dmodel,dv).to('cuda:1')


    #Layer6: GPU:1
    self.keyTransformer_6 = nn.Linear(dmodel,dk).to('cuda:1')
    self.querryTransformer_6 = nn.Linear(dmodel,dk).to('cuda:1')
    self.valueTrasformer_6 = nn.Linear(dmodel,dv).to('cuda:1')


    #Layer7: GPU:1
    self.keyTransformer_7 = nn.Linear(dmodel,dk).to('cuda:1')
    self.querryTransformer_7 = nn.Linear(dmodel,dk).to('cuda:1')
    self.valueTrasformer_7  = nn.Linear(dmodel,dv).to('cuda:1')


    #Layer8: GPU:1
    self.keyTransformer_8 = nn.Linear(dmodel,dk).to('cuda:1')
    self.querryTransformer_8 = nn.Linear(dmodel,dk).to('cuda:1')
    self.valueTrasformer_8  = nn.Linear(dmodel,dv).to('cuda:1')



    #Weight_0 layer:
    self.W0 = nn.Linear(dmodel,dmodel).to('cuda:0')   #Size h*dv x dmodel. But since dv = dk and dk x h = dv so it's a dmodel x dmodel layer -> cuda:0


    #LayerNormalisation
    self.Add_and_Nom = nn.LayerNorm((maximum_sequence_length,dmodel), eps=1e-05, elementwise_affine=True)

  def forward(self,x):
    k1 = self.keyTransformer_1(x)
    q1 = self.querryTransformer_1(x)
    v1 = self.valueTrasformer_1(x)

    k2 = self.keyTransformer_2(x)
    q2 = self.querryTransformer_2(x)
    v2 = self.valueTrasformer_2(x)

    k3 = self.keyTransformer_3(x)
    q3 = self.querryTransformer_3(x)
    v3 = self.valueTrasformer_3(x)

    k4 = self.keyTransformer_4(x)
    q4 = self.querryTransformer_4(x)
    v4 = self.valueTrasformer_4(x)

    x.to('cuda:1')

    k5 = self.keyTransformer_5(x)
    q5 = self.querryTransformer_5(x)
    v5 = self.valueTrasformer_5(x)

    k6 = self.keyTransformer_6(x)
    q6 = self.querryTransformer_6(x)
    v6 = self.valueTrasformer_6(x)

    k7 = self.keyTransformer_7(x)
    q7 = self.querryTransformer_7(x)
    v7 = self.valueTrasformer_7(x)

    k8 = self.keyTransformer_8(x)
    q8 = self.querryTransformer_8(x)
    v8 = self.valueTrasformer_8(x)


    head_1 = calculate_head(q1,k1,v1) #cuda:0
    head_1.to('cpu')                  # transfer everything back to cpu because torch.cat is faster on cpu to reduce model size on GPU

    head_2 = calculate_head(q2,k2,v2) #cuda:0
    head_2.to('cpu')                  # transfer everything back to cpu because torch.cat is faster on cpu to reduce model size on GPU

    head_3 = calculate_head(q3,k3,v3) #cuda:0
    head_3.to('cpu')                  # transfer everything back to cpu because torch.cat is faster on cpu to reduce model size on GPU

    head_4 = calculate_head(q4,k4,v4) #cuda:0
    head_4.to('cpu')                  # transfer everything back to cpu because torch.cat is faster on cpu to reduce model size on GPU

    head_5 = calculate_head(q1,k1,v1) #cuda:1
    head_5.to('cpu')                  # transfer everything back to cpu because torch.cat is faster on cpu to reduce model size on GPU

    head_6 = calculate_head(q2,k2,v2) #cuda:1
    head_6.to('cpu')                  # transfer everything back to cpu because torch.cat is faster on cpu to reduce model size on GPU

    head_7 = calculate_head(q3,k3,v3) #cuda:1
    head_7.to('cpu')                  # transfer everything back to cpu because torch.cat is faster on cpu to reduce model size on GPU

    head_8 = calculate_head(q4,k4,v4) #cuda:1
    head_8.to('cpu')                  # transfer everything back to cpu because torch.cat is faster on cpu to reduce model size on GPU   


    All_heads = torch.cat((head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8), dim=2) #in cpu

    All_heads.to('cuda:0') #Transfer back to Cuda:0 for matrix operation

    All_heads = self.W0(All_heads) # cuda:0

    x.to('cuda:0') # transfer back to cuda:0 for layer normalisation operation

    All_heads = self.Add_and_Nom(x + All_heads)  #cuda:0

    return All_heads #cuda:0

  def calculate_head(self,q,k,v):
    return torch.matmul(F.softmax(torch.matmul(q,k.transpose(-2,-1))/(dk**0.5)),v)

In [129]:
class FFN(nn.Module):
  def __init__(self,dmodel,dk,dv,maximum_sequence_length,dff):
    super(FFN,self).__init__()
    # 6 stacks in total
    self.multihead_1 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_2 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_3 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_4 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_5 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_6 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)

    self.FFN1 = nn.Linear(dmodel,dff)
    self.FFN2 = nn.Linear(dmodel,dff)
    self.FFN3 = nn.Linear(dmodel,dff)
    self.FFN4 = nn.Linear(dmodel,dff)
    self.FFN5 = nn.Linear(dmodel,dff)
    self.FFN6 = nn.Linear(dmodel,dff)

    self.add_and_norm_1 = nn.LayerNorm((dff,dmodel), eps=1e-05, elementwise_affine=True)
    self.add_and_norm_2 = nn.LayerNorm((dff,dmodel), eps=1e-05, elementwise_affine=True)
    self.add_and_norm_3 = nn.LayerNorm((dff,dmodel), eps=1e-05, elementwise_affine=True)
    self.add_and_norm_4 = nn.LayerNorm((dff,dmodel), eps=1e-05, elementwise_affine=True)
    self.add_and_norm_5 = nn.LayerNorm((dff,dmodel), eps=1e-05, elementwise_affine=True)
    self.add_and_norm_6 = nn.LayerNorm((dff,dmodel), eps=1e-05, elementwise_affine=True)

  def forward(self,x):
    sublayer1 = self.multihead_1(x)
    sublayer1 = self.FFN1(sublayer1)
    sublayer1 = F.relu(sublayer1)
    sublayer1 = self.add_and_norm_1(x+sublayer1)

    sublayer2 = self.multihead_2(sublayer1)
    sublayer2 = self.FFN2(sublayer2)
    sublayer2 = F.relu(sublayer2)
    sublayer2 = self.add_and_norm_2(sublayer1+sublayer2)

    sublayer3 = self.multihead_3(sublayer2)
    sublayer3 = self.FFN3(sublayer3)
    sublayer3 = F.relu(sublayer3)
    sublayer3 = self.add_and_norm_3(sublayer2+sublayer3)

    sublayer4 = self.multihead_4(sublayer3)
    sublayer4 = self.FFN4(sublayer4)
    sublayer4 = F.relu(sublayer4)
    sublayer4 = self.add_and_norm_4(sublayer3+sublayer4)

    sublayer5 = self.multihead_5(sublayer4)
    sublayer5 = self.FFN5(sublayer5)
    sublayer5 = F.relu(sublayer5)
    sublayer5 = self.add_and_norm_5(sublayer4+sublayer5)

    sublayer6 = self.multihead_6(sublayer5)
    sublayer6 = self.FFN6(sublayer6)
    sublayer6 = F.relu(sublayer6)
    sublayer6 = self.add_and_norm_6(sublayer5+sublayer6)

    return sublayer6